<a href="https://colab.research.google.com/github/Hutaf/Resume-Matcher/blob/master/Resume_Matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Resume matcher 
helps companies to find the ideal candidates based on job description

### Steps:
###### 1- Resume and job description parsing: convert pdf and docx files to list of string 
###### 2- Removing stop words and cleaning
###### 3- Use CountVector and cosine similarity metric to find the matching percentage 

## 1

In [ ]:
! pip install docx2txt
! pip install pdfminer3
! pip install PyPDF2
! pip install gensim
! pip install nltk

     |████████████████████████████████| 81kB 2.1MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=02bc8d757f43d7cfdc64e2b5be6c6a759232f672164bc47e0ba38f63d9876788
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [ ]:
! pip install docx
! pip install python-docx

     |████████████████████████████████| 5.5MB 2.6MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=4c4ab9d1e9e5e0ee225274c1150b7f864f8712819a72c465001e2e7cb2f5eead
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx


In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-07-11 01:31:03--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.47.118
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.47.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  62.1MB/s    in 26s     

2020-07-11 01:31:30 (59.5 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Libraries
import os
import collections
import docx2txt
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import docx 

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def convert_pdf_to_txt(path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(path, 'rb') as fh:

        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()
    converter.close()
    fake_file_handle.close()
    return text

In [ ]:
projectDir = '/content/drive/My Drive/'
resumeDir = projectDir + 'Resumes/' 
jobPostDir= projectDir + 'Job/'

In [ ]:
# Extracting Resume and any file 

def MultipleConvert(FolderDir):
 resumeTxt = []
 for filename in os.listdir(FolderDir):
    if(filename.endswith(".pdf")):
        try:
            resumeTxt.append(convert_pdf_to_txt(FolderDir+filename))
        except Exception:
            print('Error reading .pdf file' + filename)
    if(filename.endswith(".docx")):
        try:
            doc = docx.Document(FolderDir + filename)
            data = ""
            fullText = []
            for para in doc.paragraphs:
                fullText.append(para.text)
                data = '\n'.join(fullText)
            resumeTxt.append(data)
        except IOError:
            print('Error reading .docx file' + filename)
    if(filename.endswith(".doc")):
        try:
            text = textract.process(FolderDir+filename).decode('utf-8')     
            resumeTxt.append(text)
        except Exception:
            print('Error reading .doc file :' + filename)
 return resumeTxt

In [ ]:
resumeTxt = MultipleConvert(resumeDir)
JobTxt = MultipleConvert(jobPostDir)

Error reading .doc file :Allegis_Elaine Cheng_Asst FM.doc


In [ ]:
print(JobTxt[0])

 

 

JOB DESCRIPTION:  ACCOUNTANT 

 
Position  Summary:    Is  responsible  for  full  charge  bookkeeping,  including  maintaining 
company financial records, monthly billing, billable time tracking and reports for review 
by the management team and outside certified public accounting firm.  This includes, 
but  is  not  limited  to:  accounts  receivable  and  payable,  EOM  reconciliation,  payroll, 
payroll  taxes,  quarterly  payroll  tax  reports,  invoicing,  personnel  payroll  records,  paid 
time off tracking, weekly, monthly, quarterly and yearly business reports, computer data 
entry of time slips as well as working with the company’s accounting firm and insurance 
agent  and  other  items  as  needed  by  management.    Experience  in  QuickBooks  (or 
equivalent) and Microsoft Office products required.   
 
Part‐time position:  25‐28 hours per week.   
 
Reports to:  President 
 
Duties and Responsibilities:   
 

1.  Full charge bookkeeping including all month and year

In [ ]:
print(resumeTxt[0])

CHI CHUNG MING, MINGSON (池鐘銘) 

Mobile: 61992356  Email: mingsonchi@gmail.com   

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

Sep 2007–May 2011 

Sep 2005–Jun 2007 

Sep 2000–Jun 2005 

EDUCATION 

Hong Kong Shue Yan University   

 

 

- 

Bachelor of Commerce (Hons) in Accounting 

Caritas Wu Cheng-chung Secondary School  

- 

F.7 graduate 

King’s College 

- 

F.5 graduate 

 

 

 

 

 

 

PROFESSIONAL QUALIFICATIONS 

- 

HKICPA Qualification Program - Qualified 

-  普通話水平測試等級証書-三級甲等 

WORK EXPERIENCE 

Vistra Hong Kong, Senior Accountant 

 

 

 

 

 

 

 

 

 

 

Jul 2016–Present 

- 

Financial Reporting 

  Prepare financial statements according to HKSA and HKFRS 

  Review and comment on financial statements 

  Review audit report 

  Liaise with auditor 

  Communicate with client 

  Prepare Profits Tax Return for client 

- 

Payroll Processing 

  Prepare monthly payroll report 

  Prepare tax returns for employees and employ

In [ ]:
len(resumeTxt)

6

In [ ]:
len(JobTxt)

1

In [ ]:
type(resumeTxt)

list

In [ ]:
# NOT IMP
# Save Clean files into txt file
f=open('Clean_Resume.txt','w', encoding='utf-8')
for ele in resumeTxt:
    f.write(ele+'\n')
f.close()

f1=open('Clean_Job.txt','w', encoding='utf-8')
for ele in JobTxt:
    f1.write(ele+'\n')
f1.close()

## 2 

In [ ]:
from nltk.corpus import stopwords
stop_word = set(stopwords.words('english'))

In [ ]:
def Cleaning(listTxt):
 clean = []
 for x in listTxt:
    text = x
    text = text.split()
    text = [w for w in text if w not in stop_word] # Remove stop_words
    text = [w for w in text if w.isalpha()] # Remove numbers and special characters
    clean.append(" ".join(text))
 return clean

In [ ]:
# Clean resumes and job description
cleanresume = Cleaning(resumeTxt)
cleanjob = Cleaning(JobTxt)

In [ ]:
len(resumeTxt[0])

2513

In [ ]:
len(cleanresume[0])

1543

In [ ]:
len(JobTxt[0])

4376

In [ ]:
len(cleanjob[0])

2425

In [ ]:
cleanjob[0]

'JOB ACCOUNTANT Position Is responsible full charge including maintaining company financial monthly billable time tracking reports review management team outside certified public accounting This limited accounts receivable EOM payroll quarterly payroll tax personnel payroll paid time quarterly yearly business computer data entry time slips well working accounting firm insurance agent items needed Experience QuickBooks Microsoft Office products hours per Reports President Duties Full charge bookkeeping including month year end journal Compilation preparation quarterly yearly financial statements use CPA firm year end tax purposes decision Maintain employee records including limited mileage personal time Process payroll file payroll taxes forms quarterly yearly required Federal State Maintain payroll files Manage Timeslips program limited reviewing downloading checking accuracy Maintain program update including Maintain client accounts billings Collection accounts receivable Prepare mont

In [ ]:
cleanresume[0]

'CHI CHUNG MINGSON Sep Sep Sep EDUCATION Hong Kong Shue Yan University Bachelor Commerce Accounting Caritas Wu Secondary School graduate College graduate PROFESSIONAL QUALIFICATIONS HKICPA Qualification Program Qualified WORK EXPERIENCE Vistra Hong Senior Accountant Jul Financial Reporting Prepare financial statements according HKSA HKFRS Review comment financial statements Review audit report Liaise auditor Communicate client Prepare Profits Tax Return client Payroll Processing Prepare monthly payroll report Prepare tax returns employees employers Internal financial operation Assist manager prepare ad hoc reports TMF Accountant Jul Financial Reporting Prepared financial statements according HKSA HKFRS Reviewed comment financial statements Reviewed audit report Liaise auditor Payment Processing Prepared domestic oversea payment per instruction Internal financial operation Assisted supervisor prepare ad hoc reports Tricor Services Accounting Officer Sep Financial Reporting Prepared fina

## 3

just calculating cosine similarity between vectors

In [ ]:
job_description = str(cleanjob)

In [ ]:
type(job_description)

str

In [ ]:
# Extracting files names
Names = []
for filename in os.listdir(resumeDir):
    Names.append(os.path.splitext(filename)[0])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
i = 0
for x in cleanresume:
    text = [x, job_description]
    count_matrix = cv.fit_transform(text)
    #get the match percentage
    matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
    matchPercentage = round(matchPercentage, 3) 
    print("("+  Names[i] +")"+ " resume matches about "+ str(matchPercentage)+ "% of the job description.")
    i += 1

(Vistra_Mingson Chi_Sr Accountant) resume matches about 35.762% of the job description.
(Pearson_Henry Yeung_Accountant) resume matches about 36.633% of the job description.
(Yes & Co_Brian Ho_Asst Accountant) resume matches about 25.736% of the job description.
(Lynks_Sherry Chui_Sr Acc Officer) resume matches about 28.506% of the job description.
(Maurice Zhang_Edwin Jiang_Accountant) resume matches about 29.383% of the job description.
(HSBC_Johnson Chan_Trust Acc Officer) resume matches about 39.525% of the job description.
